In [ ]:
!pip install crewai langchain-groq crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.7/462.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool
from langchain_groq import ChatGroq
import os

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
llm = ChatGroq(model_name='llama3-8b-8192', api_key='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')

In [ ]:
support_agent = Agent(
    role='Senior Customer Support Representative',
    goal='Help and answer questions to {person} from {customer} in a detailed, elaborative fashion based on factual data.',
    backstory='You work at a company named LangChain (https://www.langchain.com/) where you are the senior member of the customer support team.',
    llm=llm,
    allow_delegation=False)

In [ ]:
qa_agent = Agent(
    role='Customer Support Quality Assurance Engineer',
    goal='You view the answers provided by Senior Customer Support Representative to see if he has covered everything needed by the customer in a factual manner.',
    backstory='You work at a company named LangChain (https://www.langchain.com/) where you are the customer support quality assurance engineer.',
    llm=llm)

In [ ]:
scrape_tool = ScrapeWebsiteTool(website_url='https://www.langchain.com/')

In [ ]:
support_task = Task(
    description=('{person} from {customer} has made an inquiry. Find factual data related to the inquiry and provide a detailed, elaborative response'),
    expected_output=('A detailed, elaborative response covering all the essential requirements related to the inquiry made by the user.'),
    tools=[scrape_tool],
    agent=support_agent
)

In [ ]:
qa_task = Task(
    description=('Review the response of the Senior Customer Support Representative and see if he has covered everything needed by the customer in a factual manner related to the {inquiry} by the {person} from {customer}.'),
    expected_output=('A finalised, complete, detailed answer to be sent to the user'),
    agent=qa_agent
)

In [ ]:
crew = Crew(
    agents=[support_agent, qa_agent],
    tasks=[support_task, qa_task],
    memory=True
)

In [ ]:
crew.kickoff(inputs={
    'person': 'Ali Raza',
    'customer': 'NUST',
    'inquiry': 'How do I create RAG appliations using LangChain?'
})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}